In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
data = load_data(2024, 15)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<
""",
        2028,
        None,
    ),
    (
        """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^
""",
        10092,
        9021,
    ),
]

# Part 1

In [ ]:
def parse_input(data, large):
    layout, moves = data.split("\n\n")
    walls = set()
    blocks = set()
    for j, line in enumerate(layout.splitlines()):
        for i, c in enumerate(line):
            if large:
                i = 2 * i
            if c == "#":
                walls.add((i, j))
                if large:
                    walls.add((i + 1, j))
            elif c == "O":
                blocks.add((i, j))
            elif c == "@":
                robot = i, j
            else:
                assert c == "."
    directions = {
        "v": (0, 1),
        "^": (0, -1),
        ">": (1, 0),
        "<": (-1, 0),
    }
    moves = [directions[c] for c in "".join(moves.splitlines())]
    return robot, walls, blocks, moves

In [ ]:
def print_map(robot, walls, blocks, large):
    width = max(i for i, _ in walls) + 1
    height = max(j for _, j in walls) + 1
    for j in range(height):
        line = ""
        for i in range(width):
            if (i, j) == robot:
                line += "@"
            elif (i, j) in walls:
                line += "#"
            elif (i, j) in blocks:
                if large:
                    line += "["
                else:
                    line += "O"
            elif large and (i - 1, j) in blocks:
                line += "]"
            else:
                line += "."
        print(line)

In [ ]:
def push_small_blocks(pos, dir_, blocks, walls):
    bi, bj = pos
    di, dj = dir_
    while (bi, bj) in blocks:
        bi += di
        bj += dj
    if (bi, bj) in walls:
        return blocks, False
    if (bi, bj) != pos:
        return blocks - {pos} | {(bi, bj)}, True
    return blocks, True

In [ ]:
def move_blocks(robot, walls, blocks, moves, push_blocks):
    blocks = blocks.copy()
    i, j = robot
    for di, dj in moves:
        blocks, moved = push_blocks((i + di, j + dj), (di, dj), blocks, walls)
        if moved:
            i += di
            j += dj
    return robot, blocks

In [ ]:
def move_and_locate(data, *, large=False, push_blocks=push_small_blocks):
    _, blocks = move_blocks(*parse_input(data, large=large), push_blocks)
    return sum(i + 100 * j for i, j in blocks)

In [ ]:
check(move_and_locate, tests)
move_and_locate(data)

# Part 2

In [ ]:
def push_large_blocks(pos, dir_, blocks, walls):
    di, dj = dir_
    to_move = set()
    to_check = {pos}
    while to_check:
        i, j = to_check.pop()
        if (i, j) in walls:
            return blocks, False
        for bi in (i - 1, i):
            if (bi, j) in blocks:
                to_move.add((bi, j))
                if di == 0:
                    to_check |= {(bi, j + dj), (bi + 1, j + dj)}
                else:
                    to_check |= {(i + 2 * di, j)}
    if to_move:
        return (
            blocks - to_move | {(i + di, j + dj) for i, j in to_move},
            True,
        )
    return blocks, True

In [ ]:
check(move_and_locate, tests, 2, large=True, push_blocks=push_large_blocks)
move_and_locate(data, large=True, push_blocks=push_large_blocks)